In [17]:
!pip install pyvi
import nltk
import numpy as np
import pandas as pd
from collections import Counter
from pyvi import ViTokenizer
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
import random
import pprint, time

In [18]:
!wget https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_train.txt
!wget https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_test.txt

--2021-03-16 11:22:57--  https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1664450 (1.6M) [text/plain]
Saving to: ‘vi_train.txt.1’

vi_train.txt.1      100%[===================>]   1.59M  --.-KB/s    in 0.06s   

2021-03-16 11:22:58 (26.6 MB/s) - ‘vi_train.txt.1’ saved [1664450/1664450]

--2021-03-16 11:22:58--  https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response..

In [19]:
def read_file(filename):
    text = open(filename, 'r', encoding='utf8').readlines()
    data = []
    for sent in text:
        for word_tag in sent.strip().split(' '):
            data.append(tuple(word_tag.split('/')))

    data = [word_tag for word_tag in data if len(word_tag) == 2]
    return data

In [20]:
train = read_file('vi_train.txt')
test = read_file('vi_test.txt')

In [21]:
tags = {tag for word,tag in train}
vocab = {word for word,tag in train}
tags_bag = [tag for word,tag in train]

In [22]:
def word_given_tag(word, tag, train_bag = train):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#tổng số lần thẻ đã vượt qua xảy ra trong train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    #tổng số lần từ đã chuyển xuất hiện dưới dạng thẻ đã chuyển.
    count_w_given_tag = len(w_given_tag_list)    
    return (count_w_given_tag, count_tag)


In [23]:
## dự đoán t1 theo t2
def t2_given_t1(t2, t1, train_bag = train):
    tags = tags_bag
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return count_t2_t1/ count_t1

In [9]:
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float64')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)

In [10]:
##xác xuất t2 với t1
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)


,,):,&,S,:,.),P,p,Ap,~,),…,*,.,A,I,’,Ny,NC,Eb,V,;,a,",",R,Ab,M,-,N,B,Nu,(,Y,C,?,Nb,],Np,H,T,!,“,v,”,Vb,Nc,Vy,...,+,m,L,"""",E,>
,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009804,0.000000,0.000000,0.009804,0.000000,0.00,0.068627,0.000000,0.000000,0.000000,0.009804,0.000000,0.176471,0.000000,0.529412,0.000000,0.117647,0.000000,0.009804,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068627,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
):,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
&,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.416667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.305556,0.000000,0.00,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
S,0.000000,0.000000,0.000000,0.000000,0.007042,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.098592,0.000000,0.000000,0.007042,0.000000,0.00,0.190141,0.000000,0.000000,0.028169,0.028169,0.000000,0.000000,0.000000,0.591549,0.000000,0.000000,0.000000,0.000000,0.007042,0.000000,0.000000,0.000000,0.014085,0.000000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,0.007042,0.000000,0.007042,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
:,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013008,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014634,0.000000,0.000000,0.005691,0.000000,0.00,0.079675,0.000000,0.000000,0.000000,0.017073,0.000000,0.025203,0.001626,0.145528,0.000813,0.000000,0.000000,0.000000,0.001626,0.000000,0.000813,0.000000,0.028455,0.000000,0.001626,0.000000,0.578049,0.000000,0.001626,0.000000,0.017886,0.000000,0.000000,0.000000,0.000000,0.014634,0.028455,0.023577,0.000000
.),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
P,0.000120,0.000000,0.001807,0.000120,0.006745,0.000000,0.028303,0.000000,0.000000,0.000000,0.001204,0.000361,0.000000,0.052511,0.040708,0.000361,0.000000,0.005058,0.000000,0.00,0.315789,0.000241,0.000000,0.082019,0.200169,0.000000,0.018668,0.001807,0.078646,0.000000,0.000482,0.003372,0.000000,0.047091,0.010358,0.000241,0.000000,0.006383,0.000000,0.010960,0.004697,0.002409,0.000000,0.010358,0.000000,0.010237,0.000000,0.008069,0.000000,0.000000,0.009876,0.000602,0.040226,0.000000
p,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [11]:
## tìm trạng thái ẩn có khả năng xảy ra cao nhất
def Viterbi(words, train_bag = train):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))


In [12]:
# lấy ra vài từ trong tập test
minitest = test[:500]
w = [w for w, t in minitest]

In [13]:
dudoan = Viterbi(w)
dudoan

[('Trên', 'E'),
 ('đường', 'N'),
 ('xuất_hiện', 'V'),
 ('nhiều', 'A'),
 ('cặp', 'M'),
 ('cha', 'N'),
 ('-', '-'),
 ('con', 'Nc'),
 (',', ','),
 ('mẹ', 'N'),
 ('-', '-'),
 ('con', 'Nc'),
 (',', ','),
 ('hay', 'C'),
 ('có_khi', 'R'),
 ('là', 'V'),
 ('cả', 'T'),
 ('nhà', 'N'),
 (',', ','),
 ('tay_xách_nách_mang', 'Ny'),
 ('vừa', 'R'),
 ('đi', 'V'),
 ('vừa', 'R'),
 ('dò', 'V'),
 ('bản_đồ', 'N'),
 ('.', '.'),
 ('Tại', 'E'),
 ('vũ_trường', 'N'),
 ('N', 'Y'),
 ('nổi_tiếng', ''),
 (',', ''),
 ('dân_chơi', 'N'),
 ('gọi', 'V'),
 ('rượu', 'N'),
 ('uống', 'V'),
 ('không', 'R'),
 ('hết', 'V'),
 ('thì', 'C'),
 ('gửi', 'V'),
 (',', ','),
 ('riêng', 'A'),
 ('Yến', 'Np'),
 ('...', '...'),
 ('bỏ', 'V'),
 ('luôn', 'R'),
 (',', ','),
 ('mai', 'N'),
 ('thích', 'V'),
 ('gọi', 'V'),
 ('tiếp', 'V'),
 ('.', '.'),
 ('Xét_nghiệm', ''),
 (',', ''),
 ('uống', 'V'),
 ('thuốc', 'N'),
 (',', ','),
 ('phản_ứng', 'V'),
 ('thuốc', 'N'),
 (',', ','),
 ('sốt', 'V'),
 ('cao', 'A'),
 (',', ','),
 ('sụt', 'V'),
 ('cân', 'V')

In [14]:
cauthuc = minitest
cauthuc

[('Trên', 'E'),
 ('đường', 'N'),
 ('xuất_hiện', 'V'),
 ('nhiều', 'A'),
 ('cặp', 'N'),
 ('cha', 'N'),
 ('-', '-'),
 ('con', 'N'),
 (',', ','),
 ('mẹ', 'N'),
 ('-', '-'),
 ('con', 'N'),
 (',', ','),
 ('hay', 'C'),
 ('có_khi', 'Ny'),
 ('là', 'C'),
 ('cả', 'T'),
 ('nhà', 'N'),
 (',', ','),
 ('tay_xách_nách_mang', 'Ny'),
 ('vừa', 'R'),
 ('đi', 'V'),
 ('vừa', 'R'),
 ('dò', 'V'),
 ('bản_đồ', 'N'),
 ('.', '.'),
 ('Tại', 'E'),
 ('vũ_trường', 'N'),
 ('N', 'Ny'),
 ('nổi_tiếng', 'A'),
 (',', ','),
 ('dân_chơi', 'N'),
 ('gọi', 'V'),
 ('rượu', 'N'),
 ('uống', 'V'),
 ('không', 'R'),
 ('hết', 'V'),
 ('thì', 'C'),
 ('gửi', 'V'),
 (',', ','),
 ('riêng', 'A'),
 ('Yến', 'Np'),
 ('...', '...'),
 ('bỏ', 'V'),
 ('luôn', 'R'),
 (',', ','),
 ('mai', 'N'),
 ('thích', 'V'),
 ('gọi', 'V'),
 ('tiếp', 'V'),
 ('.', '.'),
 ('Xét_nghiệm', 'V'),
 (',', ','),
 ('uống', 'V'),
 ('thuốc', 'N'),
 (',', ','),
 ('phản_ứng', 'V'),
 ('thuốc', 'N'),
 (',', ','),
 ('sốt', 'V'),
 ('cao', 'A'),
 (',', ','),
 ('sụt', 'V'),
 ('cân', 

In [15]:
def danhgia(tagthuc, tagdudoan):
    count = 0
    for i in range(len(tagthuc)):
        if tagthuc[i] == tagdudoan[i]:
            count +=1
    return count/len(tagthuc)*100

print('Accuracy:',danhgia(cauthuc,dudoan))


Accuracy: 86.0


In [16]:
sent = 'Dù khá đắt nhưng tôi vẫn đồng ý.'
Viterbi(ViTokenizer.tokenize(sent).split())

[('Dù', 'C'),
 ('khá', 'R'),
 ('đắt', 'A'),
 ('nhưng', 'C'),
 ('tôi', 'P'),
 ('vẫn', 'R'),
 ('đồng_ý', 'V'),
 ('.', '.')]